# Transporter analysis of bacillus mother-spore

In [1]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join, dirname, abspath
import sys
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from cobrame.util.helper_functions import *

import copy
from scipy import stats
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(bacillusme.__file__)
ecoli_files = dirname(abspath(bacillusme.__file__))
pd.set_option('display.max_colwidth', None)


/home/jt/work/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/work/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/work/cobrapy-0.5.11/cobra/__init__.py
/home/jt/work/cobrame/cobrame/__init__.py
/home/jt/work/sporeme/bacillusme/__init__.py


### PORE

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
with open("./sporeme_solution_v4_proteomics_spoAQ.pickle", "rb") as outfile:
    sporeme = pickle.load(outfile)

In [7]:
from bacillusme.util.helper_functions import get_all_transport_of_model
transport = get_all_transport_of_model(sporeme)

100%|██████████| 11244/11244 [00:00<00:00, 519666.28it/s]


In [25]:
tmp = [r for r in transport if "DM_" not in r and "trans" not in r and "formation" not in r and "demand" not in r and "_s" in r]
fluxes = pandas.Series(sporeme.solution.x_dict)[tmp]
fluxes[fluxes!=0].shape[0]

49

In [62]:
flux_based_reactions(sporeme,'PORE-CPLX_s',growth_symbol = 'sigma')

,lb,ub,rxn_flux,met_flux,reaction
formation_combined_PORE-CPLX_s,0,1000,2.96016e-06,2.96016e-06,12.0 protein_BSU24360_Inner_Membrane + 30.0 protein_BSU24370_Inner_Membrane + protein_BSU24380_Inner_Membrane + protein_BSU24390_Inner_Membrane + protein_BSU24400_Inner_Membrane + protein_BSU24410_Inner_Membrane + protein_BSU24420_Inner_Membrane + 6.0 protein_BSU24430_Inner_Membrane + 12.0 protein_BSU36550_Inner_Membrane_s --> PORE-CPLX_s
2PGt6_FWD_PORE-CPLX_s,0,999999,6.13171,-3.78697e-07,2pg_c + h_c --> 2pg_s + -4.27350427350427e-6*sigma PORE-CPLX_s + h_s
CO2t_REV_PORE-CPLX_s,0,999999,4.90016,-3.02636e-07,co2_s --> -4.27350427350427e-6*sigma PORE-CPLX_s + co2_c
L_LACt2r_REV_PORE-CPLX_s,0,999999,4.22009,-2.60635e-07,h_s + lac__L_s --> -4.27350427350427e-6*sigma PORE-CPLX_s + h_c + lac__L_c
GSNt2_FWD_PORE-CPLX_s,0,999999,4.05551,-2.5047e-07,gsn_c + h_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + gsn_s + h_s
...,...,...,...,...,...
CYOR3m_FWD_PORE-CPLX_s,0,999999,0,-0,2.0 ficytcc_s + h_s + mql7_s --> -4.27350427350427e-6*sigma PORE-CPLX_s + 2.0 focytcc_s + 3.0 h_c + mqn7_s
GSNt2_REV_PORE-CPLX_s,0,999999,0,-0,gsn_s + h_s --> -4.27350427350427e-6*sigma PORE-CPLX_s + gsn_c + h_c
DIPEPabc12_FWD_PORE-CPLX_s,0,999999,0,-0,atp_s + h2o_s + met_L_ala__L_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + adp_s + h_s + met_L_ala__L_s + pi_s
SUCCt2r_FWD_PORE-CPLX_s,0,999999,0,-0,h_c + succ_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + h_s + succ_s


In [63]:
for i in sporeme.reactions.get_by_id('formation_combined_PORE-CPLX_s').reaction.split('+'):
    print(i)

12.0 protein_BSU24360_Inner_Membrane 
 30.0 protein_BSU24370_Inner_Membrane 
 protein_BSU24380_Inner_Membrane 
 protein_BSU24390_Inner_Membrane 
 protein_BSU24400_Inner_Membrane 
 protein_BSU24410_Inner_Membrane 
 protein_BSU24420_Inner_Membrane 
 6.0 protein_BSU24430_Inner_Membrane 
 12.0 protein_BSU36550_Inner_Membrane_s --> PORE-CPLX_s


In [64]:
locus_to_name = pd.read_csv('../gene_name_dictionary.csv',index_col=0)['name'].to_dict()
name_to_locus = pd.read_csv('../gene_name_dictionary.csv',index_col=1)['locus_id'].to_dict()

### Sigmas

In [65]:
l = ['sigKc','sigKn','sigE','sigF','sigG']
d = {}
for j in l:
    i = name_to_locus[j]
    rfs = 'translation_{}_s'.format(i)
    if not sporeme.reactions.has_id(rfs):continue
    rmc = 'translation_{}'.format(i)
    
    d[j] = {
        'MC' : sporeme.solution.x_dict[rmc],
        'FS' : sporeme.solution.x_dict[rfs]
    }
(pd.DataFrame.from_dict(d)*1e9).T

,MC,FS
sigKc,0.012324,0.000000
sigKn,0.012324,0.000000
sigE,124.265906,0.000000
sigF,0.000000,7.543542


In [ ]:
l = [i for i in name_to_locus if 'spoIIIA' in i] + ['spoIIQ']
d = {}
for j in l:
    i = name_to_locus[j]
    rfs = 'translation_{}_s'.format(i)
    if not sporeme.reactions.has_id(rfs):continue
    rmc = 'translation_{}'.format(i)
    
    d[j] = {
        'MC' : sporeme.solution.x_dict[rmc],
        'FS' : sporeme.solution.x_dict[rfs]
    }
(pd.DataFrame.from_dict(d)*1e9).sort_index(True).T